# Imports and configuration

In [3]:
import asyncio
import os
import pandas as pd
from typing import Dict
import sys

from aiofile import AIOFile
from aiohttp import ClientSession
from bs4 import Comment
from src.processing.coh_metrix_indices.descriptive_indices import get_paragraph_count_from_text
from src.processing.coh_metrix_indices.descriptive_indices import get_sentence_count_from_text
from src.processing.coh_metrix_indices.descriptive_indices import get_word_count_from_text
from src.processing.coh_metrix_indices.descriptive_indices import get_mean_of_length_of_paragraphs
from src.processing.coh_metrix_indices.descriptive_indices import get_std_of_length_of_paragraphs
from src.processing.coh_metrix_indices.descriptive_indices import get_mean_of_length_of_sentences
from src.processing.coh_metrix_indices.descriptive_indices import get_std_of_length_of_sentences
from src.processing.coh_metrix_indices.descriptive_indices import get_mean_of_length_of_words
from src.processing.coh_metrix_indices.descriptive_indices import get_std_of_length_of_words

# Constantes

In [4]:
PROJECT_PATH = '/home/hans/Documentos/Tesis_Chatbot'

# Helper functions

In [5]:
async def txt_file_to_pandas_dataframe_row_only_difficulty(txt_path: str, difficulty: int) -> Dict:
    """
    This function finds all the coh-metrix indices we are going to use

    Parameters:
    txt_path(str): The path of the file to obtain the indices from
    difficulty(int): The difficulty of the text
    """
    async with AIOFile(txt_path, 'r') as text_file:
        text = await text_file.read()
        return {
            'DESPC': get_paragraph_count_from_text(text), 
            'DESSC': get_sentence_count_from_text(text), 
            'DESWC': get_word_count_from_text(text), 
            'DESPL': get_mean_of_length_of_paragraphs(text), 
            'DESPLd': get_std_of_length_of_paragraphs(text),
            'DESSL': get_mean_of_length_of_sentences(text), 
            'DESSLd': get_std_of_length_of_sentences(text),
            'DESWLlt': get_mean_of_length_of_words(text), 
            'DESWLltd': get_std_of_length_of_words(text), 
            'Difficulty': difficulty
        }

In [6]:
async def convert_txt_files_to_dataframes() -> pd.DataFrame:
    """
    This function scans all the files we just downloaded and converts them to a .txt format for easier processing
    """
    levels = ['1', '2', '3']
    categories = ['Historia, Geografía y Economía',
                    'CTA',
                    'Arte',
                    'Comunicación',
                    'Personal social',
                    'Formación Ciudadana y Cívica',
                    'Educación Fisica',
                    'Educación Religiosa']
    dataframe_columns = ['DESPC', 'DESSC', 'DESWC', 'DESPL', 'DESPLd', 'DESSL', 'DESSLd' 'DESWLlt', 'DESWLltd', 'Difficulty']
    dataframe_only_difficulty = pd.DataFrame(columns=dataframe_columns)
    for level in levels:
        for category in categories:
            documents = [file_name for file_name in os.listdir(PROJECT_PATH + f'/data/raw/txt/{level}/{category}') if '.txt' in file_name]
            documents.sort()
            for document in documents:
                try:
                    txt_path = f'{PROJECT_PATH}/data/raw/txt/{level}/{category}/{document}'
                    row = await txt_file_to_pandas_dataframe_row_only_difficulty(txt_path, level)
                    dataframe_only_difficulty.append(row, ignore_index=True)
                    print(f'Text {document} converted  txt to dataframe row')
                except Exception as e:
                    print(str(e))
                    continue
    dataframe_only_difficulty.to_csv(f'{PROJECT_PATH}/data/processed/data_with_difficulty.csv')

In [7]:
async def main():
    df = await convert_txt_files_to_dataframes()
    print(df)

In [ ]:
await main()

Text abecedario.txt converted  txt to dataframe row
Text antologia-de-poesia-para-ninos-y-ninas.txt converted  txt to dataframe row
Text aprendemos-jugando-2018-4.txt converted  txt to dataframe row
Text aprendemos-jugando-2018-5.txt converted  txt to dataframe row
Text capacidades_comunica_ama.txt converted  txt to dataframe row
Text con-los-ojos-abierto-yo-escucho.txt converted  txt to dataframe row
Text con-los-ojos-abiertos-yo-veo.txt converted  txt to dataframe row
Text cuidados-con-amor.txt converted  txt to dataframe row
Text el-muneco-de-brea.txt converted  txt to dataframe row
Text el-valor-educativo-de-la-observacion-del-desarrollo-del-nino.txt converted  txt to dataframe row
Text el-viaje-al-cielo.txt converted  txt to dataframe row
Text el-zorro-que-devoro-la-nube.txt converted  txt to dataframe row
Text entorno-educativo-calidad-educacion-inicial-guia-docentes.txt converted  txt to dataframe row
Text espacios-educativos-para-ninos-ninas-0-3.txt converted  txt to dataframe 

Text arguedas.txt converted  txt to dataframe row
Text aventuras_robinson_crusoe.txt converted  txt to dataframe row
Text aventuras_tom_sawyer.txt converted  txt to dataframe row
Text bella_bestia.txt converted  txt to dataframe row
Text bibliotecas_escolares.txt converted  txt to dataframe row
Text cartas_sor_juana.txt converted  txt to dataframe row
Text castillo.txt converted  txt to dataframe row
Text cesar_vallejo.txt converted  txt to dataframe row
Text charles_dickens.txt converted  txt to dataframe row


/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Text clorinda_matto.txt converted  txt to dataframe row
[E088] Text of length 2521557 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.
Text corsario_negro.txt converted  txt to dataframe row
Text cuadernillo_salida4_escritura_5to_grado.txt converted  txt to dataframe row
Text cuaderno-nivelacion-competencias-com-vi.txt converted  txt to dataframe row
Text cumbres_borrascosas.txt converted  txt to dataframe row
Text extrano_caso_doctor_hyde.txt converted  txt to dataframe row
Text frankenstein.txt converted  txt to dataframe row
Text guia_produccion_audiolibros.txt converted  txt to dataframe row
[E088] Text of length 1729789 exceeds max